In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)  # Set to None for all rows

df = pd.read_csv('glassdoor_salary.csv')



In [2]:
df.fillna(-1)
df['Founded'] = df['Founded'].replace( '--',-1).astype(int)



In [3]:
df = df[~df.eq('-1').any(axis=1)]

In [4]:
salary = df['Salary Estimate'].apply(lambda x : x.split('(')[0])
df['hourly']  = df['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else  0)
df['employer provided']  = df['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary' in x.lower() else  0)

In [5]:
minus_kd = salary.apply(lambda x : x.replace('K', '').replace('$', ''))
min_hr = minus_kd.apply(lambda x: x.lower().replace('per hour', '').replace('usd', ''))
min_hr = min_hr[min_hr.str.contains('-')]

In [6]:
df['min_salary'] = min_hr.apply(lambda x : x.split('-')[0])
df['max_salary'] = min_hr.apply(lambda x:x.split('-')[1])
df = df.dropna(subset = ['max_salary', 'min_salary'])

In [7]:
df['max_salary'] = df['max_salary'].str.strip()  # Remove leading/trailing spaces
df['max_salary'] = pd.to_numeric(df['max_salary'], errors='coerce')  # Convert to numeric (replace NaN with a placeholder)
df['min_salary'] = df['min_salary'].str.strip()  # Remove leading/trailing spaces
df['min_salary'] = pd.to_numeric(df['min_salary'], errors='coerce')  # Convert to numeric (replace NaN with a placeholder)
df['min_salary'] = df['min_salary'].fillna(-1).astype(int)  # Fill NaN with -1 and convert to int
df['max_salary'].astype(int)
df['min_salary'].astype(int)
df['avg_salary'] = (df.min_salary + df.max_salary)/2
df = df[df['Location'].str.contains(',')]

In [8]:
df['job_state'] = df['Location'].apply( lambda x : x.split(',')[1])
df['city'] = df['Location'].apply(lambda x : x.split(',')[0])

In [9]:

df['age'] = df.Founded.apply(lambda x : x if x < 1 else 2024-x)

In [14]:
df = df[~df.eq(-1).any(axis=1)]

In [18]:
df.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,...,Sector,Revenue,hourly,employer provided,min_salary,max_salary,avg_salary,job_state,city,age
0,0,"Manager, Model Validation",USD 76K - 126K (Glassdoor est.),"5 years of experience in statistical modeling,...",4.1,Sallie Mae,"Newark, DE",1001 to 5000 Employees,1973,Company - Public,...,Financial Services,$1 to $5 billion (USD),0,0,76,126.0,101.0,DE,Newark,51
2,2,Sr. Finance & Data Analyst,USD 71K - 101K (Glassdoor est.),PACE is an alternative to nursing home care an...,3.4,Senior Care Partners PACE,"Battle Creek, MI",201 to 500 Employees,2008,Nonprofit Organization,...,Healthcare,$5 to $25 million (USD),0,0,71,101.0,86.0,MI,Battle Creek,16
4,4,Principal Data Scientist,USD 122K - 221K (Employer est.),The analyst must be able to develop Python and...,3.8,Leidos,"Huntsville, AL",10000+ Employees,1969,Company - Public,...,Aerospace & Defense,$10+ billion (USD),0,0,122,221.0,171.5,AL,Huntsville,55
5,5,Data Scientist,USD 78K - 115K (Glassdoor est.),"And conduct what-if scenario analysis, for fie...",3.5,Penske Truck Leasing and Logistics,"Reading, PA",10000+ Employees,1969,Company - Private,...,Transportation & Logistics,$5 to $10 billion (USD),0,0,78,115.0,96.5,PA,Reading,55
25,25,Senior Data Scientist,USD 96K - 130K (Glassdoor est.),Perform technical risk analysis and reliabilit...,4.0,Deloitte,"Mechanicsburg, PA",10000+ Employees,1850,Company - Private,...,Management & Consulting,$10+ billion (USD),0,0,96,130.0,113.0,PA,Mechanicsburg,174


In [20]:
df['python_yn'] = df['Job Description'].apply(lambda x : 1 if 'python' in x.lower() else 0)
df['r_yn'] = df['Job Description'].apply(lambda x : 1 if 'r studio' in x.lower() or 'r-studio' in x.lower() else 0)

In [22]:

df['python_yn'].value_counts()


python_yn
0    557
1     24
Name: count, dtype: int64

In [ ]:
#Get Job description 
#Get spark, python, R
